# Interferograms

Prysm offers rich features for analysis of interferometric data. Interferogram objects are conceptually similar to [Pupils](./Pupils.ipynb) and both inherit from the same base class, as they both have to do with optical phase. The construction of an `Interferogram` requires only a few parameters:

In [ ]:
import numpy as np
from prysm import Interferogram
x = y = np.arange(129)
z = np.random.uniform((128,128))
interf = Interferogram(phase=z, intensity=None, x=x, y=y, scale='mm', phase_unit='nm', meta={'wavelength': 632.8e-9})

Notable are the scale, and phase unit, which define the x, y, and z units. Any SI unit is accepted as well as angstroms. Imperial units not accepted. meta is a dictionary to store metadata. For several interferogram methods to work, the wavelength must be present in meters. This departure from `prysm`’s convention of preferred units is used to maintain compatability with Zygo dat files. Interferograms are usually created from such files:

In [ ]:
from prysm import sample_files

interf = Interferogram.from_zygo_dat(sample_files('dat'))

and both the dat and datx format from Zygo are supported. Dat carries no dependencies, while datx requries the installation of `h5py`. In addition to properties inherited from the OpticalPhase class (`pv`, `rms`, `Sa`, `std`), Interferograms have a `PVr` property, for C. Evan's Robust PV metric, and `dropout_percentage` property, which gives the percentage of NaN values within the phase array. These NaNs may be filled,

In [ ]:
interf.fill(_with=0)

with 0 as a default value; only constants are supported. The modification is done in-place and the method returns self. Piston, tip-tilt, and power may be removed:

In [ ]:
interf.remove_piston().remove_tiptilt().remove_power()

again done in-place and returning self, so methods can be chained. You should remove these terms (or, indeed do anything with Zernikes) before filling NaNs.  One line convenience wrappers exist:

In [ ]:
interf.remove_piston_tiptilt()
interf.remove_piston_tiptilt_power()

spikes may also be clipped,

In [ ]:
interf.spike_clip(nsigma=3)  # default is 3

setting points with a value more than nsigma standard deviations from the mean to NaN.

If the data did not have a lateral calibration baked into it, you can provide one in `prysm`,

In [ ]:
# this is not going to do what you want if your data is already calibrated.
interf.latcal(plate_scale=0.1, unit='mm')

Masks may be applied:

In [ ]:
your_mask = np.ones(interf.phase.shape)
interf.mask(your_mask)
interf.mask('circle', diameter=4)  # 4 <spatial_unit> diameter circle

The truecircle mask should not be used on interferometric data. the phase is deleted (replaced with NaN) wherever the mask is equal to zero.

Interferograms may be cropped, deleting empty (NaN) regions around a measurment;

In [ ]:
interf.crop()

Convenience properties are provided for data size,

In [ ]:
interf.shape, interf.size, interf.diameter_x, interf.diameter_y, interf.diameter, interf.semidiameter

`shape` and `size` mirrors the underlying `interf.phase` ndarray. The x and y diameters are in units of `interf.spatial_unit` and `diameter` is the greater of the two.

The two dimensional Power Spectral Density (PSD) may be computed. The data may not contain NaNs, and piston tip and tilt should be removed prior. A 2D Welch window is used for circular data and a 2D Hanning window for rectangular data, so there is no need for concern about zero values creating a discontinuity at the edge of circular or other nonrectangular apertures.

In [ ]:
interf.crop().remove_piston_tiptilt_power().fill()
ux, uy, psd = interf.psd()

several slices are also available,

In [ ]:
psd_dict = interf.psd_slices(x=True, y=True, azavg=True, azmin=True, azmax=True)
ux, psd_x = psd_dict['x']
uy, psd_y = psd_dict['y']
ur, psd_r = psd_dict['azavg']

and the PSD may be plotted,

In [ ]:
interf.plot_psd2d(axlim=1, interp_method='lanczos', fig=None, ax=None)
interf.plot_psd_slices(xlim=(1e0,1e3), ylim=(1e-7,1e2), fig=None, ax=None)

For the slices plot, a Lorentzian or power law model may be plotted alongside the data for e.g. visual verification of a requirement:

In [ ]:
interf.plot_psd_slices(a=1,b=2,c=3)
interf.plot_psd_slices(a=1,b=1.5)

A bandlimited RMS value derived from the 2D PSD may also be evaluated,

In [ ]:
interf.bandlimited_rms(wllow=1, wlhigh=10, flow=1, fhigh=10)

only one of wavelength (wl; spatial period) or frequency (f) should be provided. f will clobber wavelength.

In [ ]:
interf.plot2d()